In [1]:
import re
import os
import pickle
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from nltk import ngrams
from collections import Counter
import random
from markov_autocomplete.autocomplete import Autocomplete

In [ ]:
nationality = ' United Kingdom '
engReviews = hotelReviews.loc[hotelReviews['Reviewer_Nationality'] == nationality][['Positive_Review','Negative_Review']]
allReviews = list(engReviews['Positive_Review'].append(engReviews['Negative_Review']))

In [ ]:
def preprocess(rev, stop_words = stopwords.words('English'), punctuations="""!"#$%&\'()*+,./:;<=>?@[\\]^_{|}~"""):
    rev = re.sub('[' + punctuations + ']', ' ', rev).strip()
    s = [item for sublist in [rev.lower().split()] for item in sublist]
    s = [x for x in s if x not in stop_words]
    s = ' '.join(s)
    return s

In [ ]:
processedReviews = []
i = 0
for ar in allReviews:
    if i % (len(allReviews) / 10) == 0:
        print 100.0 * i / len(allReviews)
    i+=1
    processedReviews.append(preprocess(ar))

In [34]:
random.shuffle(processedReviews)
train = processedReviews[200:]
test = processedReviews[:200]
testValues = getTestValues(test, 1)
len(testValues)

1783

In [35]:
with open('processedReviews.pkl', 'wb') as f:
    pickle.dump(processedReviews, f)

with open('processedTrain.pkl', 'wb') as f:
    pickle.dump(train, f)
    
with open('processedTest.pkl', 'wb') as f:
    pickle.dump(test, f)

In [6]:
with open('processedReviews.pkl', 'rb') as f:
    processedReviews = pickle.load(f)
    
with open('processedTrain.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('processedTest.pkl', 'rb') as f:
    test = pickle.load(f)

In [2]:
def getTestValues(sentences, grams = 1):

    testValues = []

    for s in sentences:
        if grams == 1:
            oneGrams = ac.get_ngrams(s,1)
            for o in range(len(oneGrams) - 1):
                testValues.append([oneGrams[o], oneGrams[o+1]])
        if grams == 2:
            twoGrams = ac.get_ngrams(s,2)
            for o in range(len(twoGrams) - 2):
                testValues.append([twoGrams[o], twoGrams[o+1].split()[-1]])
        if grams == 3:
            threeGrams = ac.get_ngrams(s,3)
            for o in range(len(threeGrams) - 3):
                testValues.append([threeGrams[o], threeGrams[o+1].split()[-1]])

    return testValues

In [37]:
with open('processedTrain.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('processedTest.pkl', 'rb') as f:
    test = pickle.load(f)

ac = Autocomplete(model_path="ngram_traintest10",
                  sentences=train,
                  n_model=3,
                  n_candidates=20,
                  match_model="start",
                  min_freq=0,
                  punctuations="",
                  lowercase=True)


testValues = getTestValues(test, 1)
predictionSuccess = []
index = 0
for t in testValues:
    if index % (len(testValues) / 200) == 0:
        print 100.0 * index / len(testValues)
    index+=1
    
    p,v = ac.predictions(t[0])
    pPos = [i for i in range(len(p)) if t[1] in p[i]]
    if(len(pPos) == 0):
        predictionSuccess.append(-1)
    else:
        predictionSuccess.append(pPos[0])

Saving the 1-grams in ngram_traintest10/1-grams.pickle
Saving the 2-grams in ngram_traintest10/2-grams.pickle
Saving the 3-grams in ngram_traintest10/3-grams.pickle
0.0
0.448681996635
0.89736399327
1.3460459899
1.79472798654
2.24340998317
2.69209197981
3.14077397644
3.58945597308
4.03813796971
4.48681996635
4.93550196298
5.38418395962
5.83286595625
6.28154795289
6.73022994952
7.17891194616
7.62759394279
8.07627593943
8.52495793606
8.9736399327
9.42232192933
9.87100392597
10.3196859226
10.7683679192
11.2170499159
11.6657319125
12.1144139091
12.5630959058
13.0117779024
13.460459899
13.9091418957
14.3578238923
14.806505889
15.2551878856
15.7038698822
16.1525518789
16.6012338755
17.0499158721
17.4985978688
17.9472798654
18.395961862
18.8446438587
19.2933258553
19.7420078519
20.1906898486
20.6393718452
21.0880538418
21.5367358385
21.9854178351
22.4340998317
22.8827818284
23.331463825
23.7801458216
24.2288278183
24.6775098149
25.1261918116
25.5748738082
26.0235558048
26.4722378015
26.9209197

In [38]:
1.0*sum(np.array(predictionSuccess) != -1) / len(predictionSuccess)

0.3583847448121144

In [39]:
np.save('OneGrams_TRTE10',predictionSuccess)

In [ ]:
testsentences = ['''What an amazing experience with really over the top friendly staff!\
                This was one of my first stays with Marriott and they really did everything right.\
                Special bedding orders - everything was arranged.\
                Room size wishes - I was upgraded.\
                The room service was fast and the portion really big with reasonable prices.\
                No need to go somewhere else after a stressful day!\
                Standard decent Marriott hotel.\
                The staff are efficient and the room we had was really quite good - large, clean and comfortable beds.\
                The hotel bar - Champions - is the standard American type bar that Marriotts have.\
                The wait staff at the bar were very nice.\
                An emblematic hotel in the center of Athens.\
                This will take into account the meetings that take place in this hotel.\
                It is a great hotel with excellent modern decoration and clean spaces.\
                The staff is courteous and willing to serve you all with a smile.\
                The meeting rooms are quite large with comfortable seating and a good acoustic.\
                The food is very delicious and it is definitely worth visiting for a meal or a stay.\
                Probably one of the top hotels in Athens, with a top roof bar (Galaxy bar) overlooking the Acropolis and Lycabetus hills.\
                The hotel offers a number of restaurants and boasts an impressive outdoor swimming pool.\
                A great choice to stay while in Athens or visit for drinks or dinner.\
                What amazing views from the Terrace Suite at Four Seasons New York.\
                The staff were all friendly, courteous, and discrete. The hotel lobby, bar, and lounge were beautiful and welcoming.\
                The room in which I stayed, had spectacular views.\
                I will look forward to many returns at this particular Four Seasons Hotel, New York.\
                Super nice staff and rooms are not bad.\
                Redone suite was super nice.\
                Club room has drinks and good snacks.\
                Hotel works as a stop for a night.\
                Not sure I'd book if you want to be in the heart of Munich.\
                It does have shops and restaurants around and good access to mass transit.\
                Room was clean, and was set up nicely.\
                Pool was really nice, and the view from the pool was fantastic.\
                At this point though, not having free wifi in the rooms (even though I'm a member of the club) is completely unacceptable.\
                This is my favorite hotel when visiting London.\
                The accommodations are excellent and the staff is very friendly and efficient.\
                The location is prime with superb views and adjacent parks for walking.\
                Oh yes a near by resistance is Buckingham Palace !!\
                Superior place , enjoyed our stay to the max , the location is excellent , just few minutes walking  to public transportation , there are quite nice good food restaurants and coffee shops very near the hotel ، Lebanese and Turkish restaurants too.\
                Hotel staff and service is great and is clean.\
                Only problem is the decor in lobby and restaurant area.\
                Instead of walking into a hotel in Greece, I felt like I was walking into a hotel in China.\
                I would prefer the greek experience when in Greece.\
                They do book a very large amount of Asians.\
                I guess money talks.\
                This hotel had a nice breakfast and clean rooms.\
                Staff was also helpful.\
                The pool closes at 7.\
                Wonderful bakery across the street!''']